In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [11]:
df = spark.read.csv('classmates.csv', sep=';', header=True)

In [14]:
df.show()

+---+------+--------------------+
| id|genero|                nome|
+---+------+--------------------+
|  1|male  |Martins          ...|
|  2|male  |Carvalho         ...|
|  3|female|Rocha            ...|
|  4|male  |Cunha            ...|
|  5|female|Costa            ...|
|  6|male  |Cardoso          ...|
|  7|male  |Almeida          ...|
|  8|female|Oliveira         ...|
|  9|male  |Castro           ...|
| 10|male  |Fernandes        ...|
| 11|female|Melo             ...|
| 12|female|Martins          ...|
| 13|female|Castro           ...|
| 14|male  |Ribeiro          ...|
| 15|female|Souza            ...|
| 16|male  |Azevedo          ...|
| 17|male  |Fernandes        ...|
| 18|female|Oliveira         ...|
| 19|female|Barros           ...|
| 20|male  |Rodrigues        ...|
+---+------+--------------------+
only showing top 20 rows

